In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

# Import and Clean data
The data from [Feeding America](https://www.feedingamerica.org/) is living in CSV files in the `/data` directory. The code block below imports that data, cleans it for our use, and combines it into the `allData.csv` file.

In [4]:
# Import data from CSV files, add year as column
importDirect = True

if importDirect:
    data_frames = {}
    for year in range(2010, 2018):
        data_frames[year] = pd.read_csv('data/{}Data.csv'.format(year))
        data_frames[year]['Year'] = year

    data_frames[2018] = pd.read_csv('data/2018Data.csv',header=1)
    data_frames[2018]['Year'] = 2018


    # Create list of preferred column names
    col_names = ['FIPS', 'State', 'Food Insecurity Rate', '# Food Insecure Persons', 'Low Threshold in state', 'Low Threshold Type', 'High Threshold in state', 'High Threshold Type', '% FI <= Low Threshold', '% FI Btwn Thresholds', '$ FI > High Threshold', 'Child food insecurity rate', '# Food Insecure Children', '% food insecure children in HH w/ HH incomes below 185 FPL', '% food insecure children in HH w/ HH incomes above 185 FPL', 'Cost Per Meal', 'Weighted Annual Food Budget Shortfall', 'Year', 'County']

    # Clean data
    for key in data_frames:
        df = data_frames[key]
        df[['County', 'Long State']] = df['County, State'].str.split(',',expand=True)
        df = df.drop(['County, State', 'Long State'],axis=1)
        df.columns = col_names # rename columns
        data_frames[key] = df

    # Concatenate all datarames into one
    frames = [data_frames[key] for key in data_frames]

    df = pd.concat(frames)
    df.to_csv('allData.csv') # export into CSV file
else:
    url = 'https://raw.githubusercontent.com/wesmith4/mat210-data-viz/master/feedingAmerica/allData.csv'
    df = pd.read_csv(url, index_col = 0)
    
projections2020 = pd.read_csv('data/2020Projections.csv')

In [5]:
forReplacement = ['Food Insecurity Rate', 'Child food insecurity rate', 'Low Threshold in state', 'High Threshold in state', '% food insecure children in HH w/ HH incomes below 185 FPL', '% food insecure children in HH w/ HH incomes above 185 FPL']
for field in forReplacement:
    df[field] = df[field].str.replace('%', '')
df

,FIPS,State,Food Insecurity Rate,# Food Insecure Persons,Low Threshold in state,Low Threshold Type,High Threshold in state,High Threshold Type,% FI <= Low Threshold,% FI Btwn Thresholds,$ FI > High Threshold,Child food insecurity rate,# Food Insecure Children,% food insecure children in HH w/ HH incomes below 185 FPL,% food insecure children in HH w/ HH incomes above 185 FPL,Cost Per Meal,Weighted Annual Food Budget Shortfall,Year,County
0,1001,AL,13.4,"7,140",130.0,SNAP,185.0,Other Nutrition Program,32.7%,20.8%,46.5%,20.3,"2,980",51,49,$ 2.58,"$ 3,170,830",2010,Autauga County
1,1003,AL,13.4,"23,570",130.0,SNAP,185.0,Other Nutrition Program,34.7%,28.7%,36.6%,23.8,"9,720",59,41,$ 2.64,"$ 10,710,730",2010,Baldwin County
2,1005,AL,23.2,"6,440",130.0,SNAP,185.0,Other Nutrition Program,47.9%,17.1%,35.0%,25.8,"1,600",87,13,$ 2.53,"$ 2,804,540",2010,Barbour County
3,1007,AL,15.7,"3,550",130.0,SNAP,185.0,Other Nutrition Program,35.8%,28.8%,35.4%,24.9,"1,300",64,36,$ 2.55,"$ 1,558,200",2010,Bibb County
4,1009,AL,12.6,"7,160",130.0,SNAP,185.0,Other Nutrition Program,41.0%,30.5%,28.5%,25.4,"3,540",53,47,$ 2.50,"$ 3,081,120",2010,Blount County
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3137,56037,WY,11.7,"5,140",130,SNAP,185,Other Nutrition Program,40.6%,15.9%,43.5%,15.4,"1,820",60,41,$3.29,"$2,865,000.00",2018,Sweetwater County
3138,56039,WY,9.5,"2,200",130,SNAP,185,Other Nutrition Program,33.8%,13.9%,52.3%,8.4,370,72,29,$4.52,"$1,683,000.00",2018,Teton County
3139,56041,WY,13.5,"2,780",130,SNAP,185,Other Nutrition Program,56.4%,9.0%,34.6%,18.7,"1,140",65,36,$3.07,"$1,444,000.00",2018,Uinta County
3140,56043,WY,12.6,"1,020",130,SNAP,185,Other Nutrition Program,45.3%,16.3%,38.4%,18.4,360,70,30,$3.26,"$562,000.00",2018,Washakie County
